In [26]:
import torch
import pandas as pd
import numpy as np
import math
import os
import ray
from ray.tune.examples.pbt_transformers import utils
import logging
from typing import Dict, Optional, Tuple
from ray import tune
import wandb
from ray.tune.integration.wandb import wandb_mixin
from ray.tune.schedulers import PopulationBasedTraining
from ray.tune import CLIReporter
import json
import random
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
from transformers.file_utils import is_torch_tpu_available
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR

In [2]:
ray.shutdown()
ray.init(log_to_driver=False, ignore_reinit_error=True)

RayContext(dashboard_url='', python_version='3.9.7', ray_version='1.12.1', ray_commit='4863e33856b54ccf8add5cbe75e41558850a1b75', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': 'tcp://127.0.0.1:54449', 'raylet_socket_name': 'tcp://127.0.0.1:54945', 'webui_url': '', 'session_dir': 'C:\\Users\\youss\\AppData\\Local\\Temp\\ray\\session_2022-06-12_14-59-34_359148_9120', 'metrics_export_port': 56440, 'gcs_address': '127.0.0.1:60092', 'address': '127.0.0.1:60092', 'node_id': '62b2aac59e90720dc184691ac13ea041a895e3907231dbd30d8f42fc'})

In [3]:
df_train = pd.read_csv(r'C:\Users\youss\Downloads\Training_data.csv')
df_test = pd.read_csv(r'C:\Users\youss\Downloads\Test_data.csv')
df_validate = pd.read_csv(r'C:\Users\youss\Downloads\Validate_data.csv')

In [4]:
df_train['Overall'] = df_train['Overall'].astype(int)
df_validate['Overall'] = df_validate['Overall'].astype(int)

In [5]:
MODEL_TYPE = 'xlm-roberta-large'
tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_TYPE)
L_RATE = 5e-6
MAX_LEN = 512

NUM_EPOCHS = 2
BATCH_SIZE = 4


In [6]:
class CustomTextDataset(Dataset):
    def __init__(self, txt1, txt2, labels):
        self.labels = labels
        self.text1 = txt1
        self.text2 = txt2
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        label = self.labels[idx]
        text1 = self.text1[idx]
        text2 = self.text2[idx]
        encoded_dict = tokenizer.encode_plus(
            str(text1), str(text2),
            add_special_tokens = True,
            max_length = MAX_LEN,     
            pad_to_max_length = True,
            return_attention_mask = True,   
            return_tensors = 'pt' # return pytorch tensors
       )
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        target = torch.tensor(label, dtype=torch.long)

        sample = {"Padded Token List": padded_token_list, "Attention Mask": att_mask, "Class": target}
        return sample

In [7]:
class CustomTestDataset(Dataset):
    def __init__(self, txt1, txt2):
        self.text1 = txt1
        self.text2 = txt2
    def __len__(self):
        return len(self.text1)
    def __getitem__(self, idx):
        text1 = self.text1[idx]
        text2 = self.text2[idx]
        encoded_dict = tokenizer.encode_plus(
            str(text1), str(text2),
            add_special_tokens = True,
            max_length = MAX_LEN,     
            pad_to_max_length = True,
            return_attention_mask = True,   
            return_tensors = 'pt' # return pytorch tensors
       )
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]

        sample = {"Padded Token List": padded_token_list, "Attention Mask": att_mask}
        return sample

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [9]:
TrainData = CustomTextDataset(df_train['text1'], df_train['text2'], df_train['Overall'])
ValidateData = CustomTextDataset(df_validate['text1'], df_validate['text2'], df_validate['Overall'])

In [10]:
TestData = CustomTestDataset(df_test['text1'], df_test['text2'])

In [11]:
os.environ["WANDB_API_KEY"] = "7684e4b61f157357e7987ef38c93c0bc46fe7c24"

In [12]:
DL_DS_Train = DataLoader(TrainData, batch_size=BATCH_SIZE, shuffle=True)

In [13]:
DL_DS_Validate = DataLoader(ValidateData, batch_size=BATCH_SIZE, shuffle=True)

In [14]:
DL_DS_Test = DataLoader(TestData, batch_size=BATCH_SIZE, shuffle=False)

In [15]:
model = XLMRobertaForSequenceClassification.from_pretrained(
    MODEL_TYPE, 
    num_labels = 5,
)

# Send the model to the device.
model.to(device)

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.out

XLMRobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
     

In [14]:
model.load_state_dict(torch.load('model_base.pt'))

<All keys matched successfully>

In [17]:
# Pass a batch of train samples to the model.

batch = next(iter(DL_DS_Validate))

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


{'Padded Token List': tensor([[    0, 12717,    68,  ...,   136,  5646,     2],
         [    0,   661, 21086,  ...,   198,    88,     2],
         [    0,  5455,  3501,  ...,  2804,  1836,     2],
         [    0,  2991, 35473,  ...,   450,  2265,     2]]),
 'Attention Mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]]),
 'Class': tensor([4, 2, 1, 3])}

In [18]:
b_input_ids = batch['Padded Token List'].to(device)
b_input_mask = batch['Attention Mask'].to(device)
b_labels = batch['Class'].to(device)


Padded Token List torch.Size([4, 512])
Attention Mask torch.Size([4, 512])
Class torch.Size([4])


In [27]:
optimizer = torch.optim.AdamW(model.parameters(),
              lr = L_RATE, 
              eps = 1e-8 
            )

In [33]:
def model_init():
    return XLMRobertaForSequenceClassification.from_pretrained(
    MODEL_TYPE, 
    num_labels = 5, return_dict=True
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)


training_args = TrainingArguments(
    "test", evaluation_strategy="steps", eval_steps=500, disable_tqdm=True)


trainer = Trainer(
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=DL_DS_Train,
    eval_dataset=DL_DS_Validate,
    model_init=model_init,
    compute_metrics=compute_metrics,
)

# Default objective is the sum of all metrics
# when metrics are provided, so we have to maximize it.
trainer.hyperparameter_search(
    direction="maximize", 
    backend="ray", 
    n_trials=5 # number of trials
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/xlm-roberta-large/resolve/main/config.json from cache at C:\Users\youss/.cache\huggingface\transformers\4d7a1550c9ab8701667bc307a1213c040fcc06dc87a5e4994e72aecc0d7e0337.842c7737719967568f4691849854475018d6cf7ce21f52576bb6e0d10091bd3c
Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.0

== Status ==
Current time: 2022-06-12 16:37:09 (running for 00:00:00.36)
Memory usage on this node: 5.4/7.8 GiB
Using FIFO scheduling algorithm.
Resources requested: 1.0/8 CPUs, 0/0 GPUs, 0.0/1.21 GiB heap, 0.0/0.61 GiB objects
Result logdir: C:\Users\youss\ray_results\_objective_2022-06-12_16-37-08
Number of trials: 5/5 (4 PENDING, 1 RUNNING)
+------------------------+----------+-----------------+-----------------+--------------------+-------------------------------+----------+
| Trial name             | status   | loc             |   learning_rate |   num_train_epochs |   per_device_train_batch_size |     seed |
|------------------------+----------+-----------------+-----------------+--------------------+-------------------------------+----------|
| _objective_2363c_00000 | RUNNING  | 127.0.0.1:10616 |     5.61152e-06 |                  5 |                            64 |  8.15396 |
| _objective_2363c_00001 | PENDING  |                 |     1.56207e-05 |                  2 |        

2022-06-12 16:37:22,214	INFO trial_runner.py:803 -- starting _objective_2363c_00002
2022-06-12 16:37:22,243	INFO trial_runner.py:803 -- starting _objective_2363c_00003
2022-06-12 16:37:22,269	INFO trial_runner.py:803 -- starting _objective_2363c_00004


== Status ==
Current time: 2022-06-12 16:37:18 (running for 00:00:09.69)
Memory usage on this node: 5.0/7.8 GiB
Using FIFO scheduling algorithm.
Resources requested: 2.0/8 CPUs, 0/0 GPUs, 0.0/1.21 GiB heap, 0.0/0.61 GiB objects
Result logdir: C:\Users\youss\ray_results\_objective_2022-06-12_16-37-08
Number of trials: 5/5 (3 PENDING, 2 RUNNING)
+------------------------+----------+-----------------+-----------------+--------------------+-------------------------------+----------+
| Trial name             | status   | loc             |   learning_rate |   num_train_epochs |   per_device_train_batch_size |     seed |
|------------------------+----------+-----------------+-----------------+--------------------+-------------------------------+----------|
| _objective_2363c_00000 | RUNNING  | 127.0.0.1:10616 |     5.61152e-06 |                  5 |                            64 |  8.15396 |
| _objective_2363c_00001 | RUNNING  | 127.0.0.1:2308  |     1.56207e-05 |                  2 |        

2022-06-12 16:37:51,108	ERROR trial_runner.py:876 -- Trial _objective_2363c_00002: Error processing event.
NoneType: None
2022-06-12 16:37:51,262	ERROR trial_runner.py:876 -- Trial _objective_2363c_00003: Error processing event.
NoneType: None


...

2022-06-12 16:38:19,024	ERROR trial_runner.py:876 -- Trial _objective_2363c_00000: Error processing event.
NoneType: None


Result for _objective_2363c_00000:
  date: 2022-06-12_16-37-18
  experiment_id: 5a47f196368c4ec8874fc0b8068e1620
  hostname: Youssefs-MateBook
  node_ip: 127.0.0.1
  pid: 10616
  timestamp: 1655044638
  trial_id: 2363c_00000
  
== Status ==
Current time: 2022-06-12 16:38:19 (running for 00:01:10.21)
Memory usage on this node: 5.1/7.8 GiB
Using FIFO scheduling algorithm.
Resources requested: 1.0/8 CPUs, 0/0 GPUs, 0.0/1.21 GiB heap, 0.0/0.61 GiB objects
Result logdir: C:\Users\youss\ray_results\_objective_2022-06-12_16-37-08
Number of trials: 5/5 (4 ERROR, 1 RUNNING)
+------------------------+----------+-----------------+-----------------+--------------------+-------------------------------+----------+
| Trial name             | status   | loc             |   learning_rate |   num_train_epochs |   per_device_train_batch_size |     seed |
|------------------------+----------+-----------------+-----------------+--------------------+-------------------------------+----------|
| _objective_2

2022-06-12 16:38:26,825	ERROR trial_runner.py:876 -- Trial _objective_2363c_00001: Error processing event.
NoneType: None


Result for _objective_2363c_00001:
  date: 2022-06-12_16-37-22
  experiment_id: fbd0443278144ae289bac27abb572e1e
  hostname: Youssefs-MateBook
  node_ip: 127.0.0.1
  pid: 2308
  timestamp: 1655044642
  trial_id: 2363c_00001
  
== Status ==
Current time: 2022-06-12 16:38:26 (running for 00:01:18.00)
Memory usage on this node: 5.0/7.8 GiB
Using FIFO scheduling algorithm.
Resources requested: 0/8 CPUs, 0/0 GPUs, 0.0/1.21 GiB heap, 0.0/0.61 GiB objects
Result logdir: C:\Users\youss\ray_results\_objective_2022-06-12_16-37-08
Number of trials: 5/5 (5 ERROR)
+------------------------+----------+-----------------+-----------------+--------------------+-------------------------------+----------+
| Trial name             | status   | loc             |   learning_rate |   num_train_epochs |   per_device_train_batch_size |     seed |
|------------------------+----------+-----------------+-----------------+--------------------+-------------------------------+----------|
| _objective_2363c_00000 | E

TuneError: ('Trials did not complete', [_objective_2363c_00000, _objective_2363c_00001, _objective_2363c_00002, _objective_2363c_00003, _objective_2363c_00004])

In [22]:
%%time


# Set the seed.
seed_val = 101

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []


# For each epoch...
for epoch in range(0, NUM_EPOCHS):
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, NUM_EPOCHS))
    

    stacked_val_labels = []
    targets_list = []

    # ========================================
    #               Training
    # ========================================
    
    print('Training...')
    
    # put the model into train mode
    model.train()
    
    # This turns gradient calculations on and off.
    torch.set_grad_enabled(True)


    # Reset the total loss for this epoch.
    total_train_loss = 0

    for i, batch in enumerate(DL_DS):
        
        train_status = 'Batch ' + str(i) + ' of ' + str(len(DL_DS))
        
        print(train_status, end='\r')


        b_input_ids = batch['Padded Token List'].to(device)
        b_input_mask = batch['Attention Mask'].to(device)
        b_labels = batch['Class'].to(device)

        model.zero_grad()        


        outputs = model(b_input_ids, 
                    attention_mask=b_input_mask,
                    labels=b_labels)
        
        # Get the loss from the outputs tuple: (loss, logits)
        loss = outputs[0]
        
        # Convert the loss from a torch tensor to a number.
        # Calculate the total loss.
        total_train_loss = total_train_loss + loss.item()
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        
        
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        
        
        # Use the optimizer to update the weights.
        
        
        optimizer.step() 
        # ========================================
        #               Validation
        # ========================================
    
    print('\nValidation...')

    # Put the model in evaluation mode.
    model.eval()

    # Turn off the gradient calculations.
    # This tells the model not to compute or store gradients.
    # This step saves memory and speeds up validation.
    torch.set_grad_enabled(False)
    
    
    # Reset the total loss for this epoch.
    total_val_loss = 0
    

    for j, batch in enumerate(val_dataloader):
        
        val_status = 'Batch ' + str(j) + ' of ' + str(len(val_dataloader))
        
        print(val_status, end='\r')

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)      


        outputs = model(b_input_ids, 
                attention_mask=b_input_mask, 
                labels=b_labels)
        
        # Get the loss from the outputs tuple: (loss, logits)
        loss = outputs[0]
        
        # Convert the loss from a torch tensor to a number.
        # Calculate the total loss.
        total_val_loss = total_val_loss + loss.item()
        

        # Get the preds
        preds = outputs[1]


        # Append the labels to a numpy list
        targets_list.extend(targets_np)

        if j == 0:  # first batch
            stacked_val_preds = val_preds

        else:
            stacked_val_preds = np.vstack((stacked_val_preds, val_preds))

    
    # Calculate the validation accuracy
    y_true = targets_list
    y_pred = np.argmax(stacked_val_preds, axis=1)
    
    val_acc = accuracy_score(y_true, y_pred)
    
    
    print('Val loss:' ,total_val_loss)
    print('Val acc: ', val_acc)

    
    print('Train loss:' ,total_train_loss)

    # Save the Model
    torch.save(model.state_dict(), 'model_truncate.pt')
    


======== Epoch 1 / 2 ========
Training...


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


...

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Train loss: 1164.4085048139095
Wall time: 2d 22h 17min 32s


In [15]:
for j, batch in enumerate(DL_DS1):
        
        inference_status = 'Batch ' + str(j+1) + ' of ' + str(len(DL_DS1))
        
        print(inference_status, end='\r')

        b_input_ids = batch['Padded Token List'].to(device)
        b_input_mask = batch['Attention Mask'].to(device)


        outputs = model(b_input_ids, 
                attention_mask=b_input_mask)
        
        
        # Get the preds
        preds = outputs[0]


        # Move preds to the CPU
        preds = preds.detach().cpu().numpy()
        
        
        # Stack the predictions.

        if j == 0:  # first batch
            stacked_preds = preds

        else:
            stacked_preds = np.vstack((stacked_preds, preds))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\youss\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2251: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

NameError: name 'b_labels' is not defined